In [1]:
################################################################################################
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)와 BM25 측정하는 예시임
# => MRR 측정 말뭉치는 KorQuAD_v1.0_dev.json 말뭉치를 이용함.
# => bm25 설치 : !pip install rank_bm25
#
# 1. 검색모델 로딩
# 2. KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
# 3. 정답리스트에 대해 임베딩 벡터 생성 후 FAISS에 인덱싱함.
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
# 5. 정답리스트와 예측리스트를 가지고 MRR을 구함
#
################################################################################################
import faiss
import numpy as np
import pandas as pd
import time
import json

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging

logger = mlogging(loggername="MRR-test", logfilename="../../../log/MRR-test")
device = GPU_info()
seed_everything(111)

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
query_num = 500            # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 10              # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
use_cross_encoder = True   # cross_encoder 사용할지 유.무 (true=사용함, false=사용안함)
use_bm25 = True

logfilepath:../../../log/MRR-test_2023-02-05.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#-------------------------------------------------------------------------------------
# 1.sentence bert 모델 로딩
#-------------------------------------------------------------------------------------
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers.cross_encoder import CrossEncoder

bi_encoder_path = "bongsoo/klue-sbert-v1"

# 임베딩 벡터 폴링 모드 선택 (*아래값중 문자열로 입력함, 기본=mean)
# mean=단어 평균, max=최대값, cls=문장, 
#['mean', 'max', 'cls', 'weightedmean', 'lasttoken']
pooling_mode = 'mean'

word_embedding_model = models.Transformer(bi_encoder_path, max_seq_length=256, do_lower_case=True, tokenizer_name_or_path=bi_encoder_path)
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())  
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode=pooling_mode)  
bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])
#bi_encoder = SentenceTransformer(bi_encoder_path)
bi_encoder.to(device)

# cross-encoder 모델 로딩
if use_cross_encoder == True:
    cross_encoder_path = "bongsoo/klue-cross-encoder-v1"#"bongsoo/albert-small-kor-cross-encoder-v1"
    cross_encoder = CrossEncoder(cross_encoder_path, max_length=512, device=device)


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [3]:
#-------------------------------------------------------------------------------------------------------
# 2. KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json

# korQuad 파일을 불러옴.
jsonfile = './data/KorQuAD_v1.0_dev.json'
contexts, questions, answers, contextids, qcontextids = read_korquad_v1_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [4]:
df_contexts.head()

,context,contextid
0,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,10001
1,"""내각과 장관들이 소외되고 대통령비서실의 권한이 너무 크다"", ""행보가 비서 본연의...",10002
2,"알렉산더 메이그스 헤이그 2세(영어: Alexander Meigs Haig, Jr....",10003
3,노터데임 대학교에서 2년간 합리적으로 심각한 공부를 한 후 헤이그는 1944년 미국...,10004
4,헤이그는 닉슨 대통령이 그를 사성 장군과 육군 부참모로 진급시킬 때 집중 광선과 논...,10005


In [5]:
df_questions.head()

,question,answer,contextid
0,1989년 6월 30일 평양축전에 대표로 파견 된 인물은?,임수경,10001
1,임종석을 검거한 장소는 경희대 내 어디인가?,학생회관 건물 계단,10001
2,임종석이 조사를 받은 뒤 인계된 곳은 어딘가?,서울지방경찰청 공안분실,10001
3,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?,임종석,10001
4,임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?,여의도 농민 폭력 시위,10001


In [6]:
print(df_contexts.shape)
print(df_questions.shape)

(964, 2)
(5533, 3)


In [7]:
#---------------------------------------------------------
# 3. 임베딩 벡터 생성 후 FAISS에 인덱싱함.
#---------------------------------------------------------

# 리스트 중복 제거 (순서유지 안함)
def remove_duplicates(lst):
    return list(set(lst))

# 리스트 중복 제거 (순서유지함)
def remove_duplicates1(lst):
    return list(dict.fromkeys(lst))

#=============================================================================
# 임베딩 벡터 생성하여 FAISS에 인덱싱하고 ID와 매핑 처리하는 함수
# => IN : contextdf
# => OUT : FASSI index
#=============================================================================
def embeddingforfaiss(df):
           
    # embedding 생성(인코딩)
    start = time.time()
    embeddings = bi_encoder.encode(df.context.to_list(), show_progress_bar=True, convert_to_tensor=False)

    #float32 로 embeddings 타입 변경
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

    # instance index 생성
    index = faiss.IndexFlatL2(embeddings.shape[1])

    # id를 매핑 시켜줌 => 이때 idtype은 반드시 int64 type이어야 함.
    index = faiss.IndexIDMap2(index)
    index.add_with_ids(embeddings, df.contextid.values)

    print(f'인코딩 시간 : {time.time()-start:.4f}')
    
    return index

In [8]:
#df 임베딩 구하고 faiss에 추가함.
index = embeddingforfaiss(df_contexts)

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

인코딩 시간 : 3.3827


In [9]:
# 테스트로 여러문장 쿼리 해봄
user_query = ["파우스트_서곡", 
             "카카오_(기업)",
             "공룡의_발견",
             "미국의_정치",
             "인공지능"]

vector = bi_encoder.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=3)

#print(distance)
#print(idx)
#print([list(df[df.idx == num]['text']) for num in idx[0]])
#print([list(df[df.idx == num]['text']) for num in idx[1]])

for i, query in enumerate(user_query):
    print(f'Q: {query}')
    count = 0
    for num in idx[i]:
        context = df_contexts[df_contexts.contextid == num]['context'].values
        print(f'{num}, {context[0]}({distance[i][count]:.3f})')
        count += 1
    print('\n')

Q: 파우스트_서곡
10039, 이 지역은 대해 때문에 세상에 알려진 지 300년 밖에 되지 않았는데, 숙종 24년(1698년) 고성 군수로 있던 남택하(南宅夏)가 찾아내고 “금강산의 얼굴빛과 같다.” 하여 해금강이라 이름 붙였다. 본래 해안 암벽, 바위섬, 자연호, 모래사장, 하천이 어우러진 경승지다. 이중 개방된 곳은 삼일포와 향로봉이며, 관동 팔경의 하나인 총석정은 개방되어 있지 않다. 삼일포는 남한의 화진포와 송지호같이 석호(潟湖)이며, 총 넓이는 0.79km에 달한다. 이 호수에는 전설에 따르면 신선 또는 화랑들이 경치가 너무 좋아 3일 동안 머물고 갔기 때문에 삼일포라 한다. 봉래대에서 삼일포 전경을 볼 수 있다. 소가 누운 모양이라고 해서 와우섬이라 이름 붙은 큰 섬을 비롯해, 3개의 작은 섬이 떠있다. 또한 삼일포 기슭에는 4명의 신선이 놀고 간 것을 기념해 세웠다는 사선정터가 있다. 향로봉은 바다의 해만물상이라 불리며, 바닷가에 육지와는 거리를 두고 홀로 솟아 있는 봉우리이다. 비바람에 씻기고 바닷물에 깎이어 독특한 모양을 지니고 있다.(51.040)
10090, 서울 중구 중림동 149번지 성요셉 아파트는 1971년에 약현성당이 지은 아파트이다. 형태는 선형식 아파트로서 언덕길을 따라 길게 휘어져 있다. 이로 인해 각각의 위치에 따라 저층이 달랐다. 저층부 1개층은 상가로 이용되고 있으며 저층부 다음부터 최고층인 6층까지 주거 시설로 이용되고 있으며 1개 동으로 구성된 이 아파트는 전면부에 3개의 입구가 있다. 언덕 하단부에 설치된 첫번째 입구의 경우 2층 1가구와 그 외 전층을 출입할 수 있고, 중앙 입구는 2층 일부 가구만 출입이 가능하다. 이 외의 출입구는 3층 이상 출입이 가능하다. 성요셉 아파트는 초기 약현성당이 성당 성도들에게 아파트를 제공하기 위하여 시작한 사업이지만 이후 민간에 매각됐다. 서울시는 성요셉아파트의 특이한 건물형태로 인해 서울시 미래유산 아파트로 선정하였다.(51.945)
10524, 노르드어는 6개의 파열음이 있다

In [10]:
#----------------------------------------------------------------
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
#----------------------------------------------------------------

from tqdm.notebook import tqdm

# Query를 list로 만들고 -> query 인코딩후->검색 결과 출력

if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_query = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_questions.sample(query_num)
    df_questions = df_questions.reset_index(drop=True)  # index는 0부터 
    user_query = df_questions['question'].values.tolist()

vector = bi_encoder.encode(user_query)
distance, idx = index.search(np.array(vector).astype("float32"), k=search_k)

# 예측검색결과를 리스트로 만듬.
bi_predictions_list = []

for i, query in enumerate(tqdm(user_query)):
    bi_predictions_list.append(idx[i].tolist())
    
print(f'----------------------------------------------------------')
print(f'bi-encoder 예측:{len(bi_predictions_list)}')
print(bi_predictions_list[0:3])

# cross-encoder 사용인 경우에
# - 한번더 검색된 결과에 {쿼리, 문장} 쌍으로 만들어서 cross-encoder로 스코어 출력함.
if use_cross_encoder == True:
    # {쿼리, 문장} 쌍 만듬.
    #count = 0
    cross_predictions_list = []
    for i, predicts in enumerate(tqdm(bi_predictions_list)):
        sentence_combinations = []
        query = user_query[i]
        #count += 1
             
        for predict in predicts:  
             sentence_combinations.append([query, df_contexts[df_contexts.contextid == predict]['context'].values.tolist()[0]])
                     
        # cross-enocoder 돌려서 출력된 score 추가함.
        cross_scores = cross_encoder.predict(sentence_combinations)+1
        #print(f'*cross_scores:{cross_scores}')
        
        dec_cross_idx = reversed(np.argsort(cross_scores))
        tmp_list = []
        for idx in dec_cross_idx:
            #print(f'idx:{idx}-predicts:{predicts[idx]}')
            tmp_list.append(predicts[idx])
         
        cross_predictions_list.append(tmp_list)   
    
    print(f'----------------------------------------------------------')
    print(f'cross-encoder 예측:{len(cross_predictions_list)}')
    print(f'{cross_predictions_list[0:3]}')


  0%|          | 0/500 [00:00<?, ?it/s]

----------------------------------------------------------
bi-encoder 예측:500
[[10624, 10353, 10183, 10241, 10604, 10625, 10616, 10196, 10523, 10359], [10026, 10856, 10454, 10872, 10623, 10453, 10518, 10290, 10083, 10079], [10312, 10453, 10479, 10885, 10877, 10279, 10882, 10193, 10285, 10002]]


  0%|          | 0/500 [00:00<?, ?it/s]

----------------------------------------------------------
cross-encoder 예측:500
[[10523, 10353, 10359, 10241, 10616, 10196, 10604, 10625, 10624, 10183], [10026, 10454, 10856, 10453, 10872, 10518, 10623, 10079, 10290, 10083], [10312, 10882, 10877, 10453, 10193, 10885, 10279, 10285, 10479, 10002]]


In [11]:
# 검색된 결과를 샘플 출력해 봄
num = 0 #출력해볼 쿼리 번호

# 쿼리
print(f'Q: {df_questions["question"][num]}')

# 정답 출력
qcontextid = df_questions["contextid"][num]
print(f'정답=========================================')
print(f'contextid:{qcontextid}')
print(f'{df_contexts[df_contexts.contextid == qcontextid]["context"].values}')

print(f'예측=============================================')

if use_cross_encoder == True:
    for pcontextid in cross_predictions_list[num]:
        print(f'contextid:{df_contexts[df_contexts.contextid == pcontextid]["contextid"].values}')
        print(f'{df_contexts[df_contexts.contextid == pcontextid]["context"].values}')
else:
    for pcontextid in bi_predictions_list[num]:
        print(f'contextid:{df_contexts[df_contexts.contextid == pcontextid]["contextid"].values}')
        print(f'{df_contexts[df_contexts.contextid == pcontextid]["context"].values}')

Q: 첫 번째 문법조약은 몇 세기에 쓰여졌나?
정답=========================================
contextid:10523
['노르드어는 9개 홑소리 위치가 모두 비음화될 수 있다. 홑소리의 이음이 앞에 비음 닿소리가 위치하면서, 근처 소리에 묻히지 않을 때 홑소리의 비음화가 일어난다. 홑소리에 실린 강세 때문에 비음이 묻혀 버릴 경우, 그 홑소리의 길이가 늘어진다. 이러한 비음화는 다른 게르만 계통 언어들에서도 나타나지만, 그렇게 길게 지속되지는 않는다. 이러한 사항들은 12세기에 쓰여진 《첫 번째 문법조약》에 나와 있기에 알 수 있는 것이며, 만일 여기 보존되어 있지 않았다면 알려지지 못했을 것이다. 《첫 번째 문법조약》에서는 글자 위에 점 하나를 찍어서 비음화를 표시하도록 하고 있다. 그러나 이 표시법은 그다지 인기가 없었고 곧 무용지불이 되었다. 동노르드어에서는 11세기를 전후하여 비모음과 구모음이 대부분 융합된 것으로 생각된다. 그러나 달라르나 방언에서는 아직 구분이 이루어지고 있다. 이하 표에서는 점을 이용해 구모음과 비모음(문자 위에 틸다 부착)을 분리하고 있다.']
예측=============================================
contextid:[10523]
['노르드어는 9개 홑소리 위치가 모두 비음화될 수 있다. 홑소리의 이음이 앞에 비음 닿소리가 위치하면서, 근처 소리에 묻히지 않을 때 홑소리의 비음화가 일어난다. 홑소리에 실린 강세 때문에 비음이 묻혀 버릴 경우, 그 홑소리의 길이가 늘어진다. 이러한 비음화는 다른 게르만 계통 언어들에서도 나타나지만, 그렇게 길게 지속되지는 않는다. 이러한 사항들은 12세기에 쓰여진 《첫 번째 문법조약》에 나와 있기에 알 수 있는 것이며, 만일 여기 보존되어 있지 않았다면 알려지지 못했을 것이다. 《첫 번째 문법조약》에서는 글자 위에 점 하나를 찍어서 비음화를 표시하도록 하고 있다. 그러나 이 표시법은 그다지 인기가 없었고 곧 무용지불이 되었다. 동노르

In [12]:
#--------------------------------------------------------------------------------------------------
# 5. MRR을 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

# MRR를 구함
if use_cross_encoder == True:
    predictions_list = bi_predictions_list
    bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    logger.info(f'--------------------------------------------------------------------------')
    logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
    logger.info('*BI-MRR:{:.4f}'.format(bi_score))
    logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
    predictions_list = cross_predictions_list
    cross_ranks, cross_score = mean_reciprocal_rank(ground_truths_list, predictions_list)
    logger.info(f'---------------------------------------------------------------------------')
    logger.info('*CROSS-ENCODER:{}'.format(cross_encoder_path))
    logger.info('*CROSS-MRR:{:.4f}'.format(cross_score))
    logger.info(f'*Ranks({len(cross_ranks)}):{cross_ranks[0:20]}')
    
    # 검색 못한 계슈
    print(f'---------------------------------------------------------------------------')
    zero_count = 0
    for item in bi_ranks:
        if item == 0:
            zero_count += 1
    
    logger.info('*검색실패계수: {}/{}({:.2f}%)'.format(zero_count, len(bi_ranks), (zero_count/len(bi_ranks))*100))
    print(f'---------------------------------------------------------------------------')
    
    logger.info(f'\nBI -> CROSS---------------------------------------------------------------------------')
    df_scores = pd.DataFrame((zip(bi_ranks, cross_ranks)), columns = ['bi-rank','cross-rank'])
    logger.info(df_scores.head(20))

else:
    predictions_list = bi_predictions_list

    bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    logger.info(f'----------------------------------------------------------------------------')
    logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
    logger.info('*BI-MRR:{:.4f}'.format(bi_score))
    logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
    # 검색 못한 계슈
    logger.info(f'---------------------------------------------------------------------------')
    zero_count = 0
    for item in bi_ranks:
        if item == 0:
            zero_count += 1
    
    logger.info('*검색실패계수: {}/{}({:.2f}%)'.format(zero_count, len(bi_ranks), (zero_count/len(bi_ranks))*100))
    logger.info(f'---------------------------------------------------------------------------')

2023-02-05 10:10:50,631 - MRR-test - INFO - --------------------------------------------------------------------------
2023-02-05 10:10:50,633 - MRR-test - INFO - *BI-ENCODER:bongsoo/klue-sbert-v1
2023-02-05 10:10:50,635 - MRR-test - INFO - *BI-MRR:0.5016
2023-02-05 10:10:50,637 - MRR-test - INFO - *Ranks(500):[0.1111111111111111, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 0, 1.0, 0.25, 0.25, 0, 0.2, 0.5, 0, 0, 1.0, 0]
2023-02-05 10:10:50,638 - MRR-test - INFO - ---------------------------------------------------------------------------
2023-02-05 10:10:50,639 - MRR-test - INFO - *CROSS-ENCODER:bongsoo/klue-cross-encoder-v1
2023-02-05 10:10:50,641 - MRR-test - INFO - *CROSS-MRR:0.6983
2023-02-05 10:10:50,641 - MRR-test - INFO - *Ranks(500):[1.0, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 0.3333333333333333, 0, 1.0, 1.0, 1.0, 0, 1.0, 1.0, 0, 0, 1.0, 0]
2023-02-05 10:10:50,643 - MRR-test - INFO - *검색실패계수: 137/500(27.40%)
2023-02-05 10:10:50,644 - MRR-test - INFO - 
BI -> CROSS--------------------------

---------------------------------------------------------------------------
---------------------------------------------------------------------------


In [13]:
#------------------------------------------------------------------------------------------------
# 6. BM25 계산
# => korquad_v1.0 말뭉치를 가지고 BM25 계산하는 함수
# => for문을 2번 돌면서 BM25 2번 계산함.
#    - 처음에는(0) 해당 쿼리와 contexts에 대해 mecab 적용 후 BM25 스코어 계산, 
#    - 2번째는 mecab 적용하지 않고 계산
#------------------------------------------------------------------------------------------------
if use_bm25 == True:

    import konlpy
    from konlpy.tag import Mecab
    from rank_bm25 import BM25Okapi
    from tqdm.notebook import tqdm

    def BM25tokenizer(sent):
      return sent.split(" ")

    # 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
    # Mecab 선언
    mecab = Mecab()

    for count in range(2):
        mecab_str = ''
        # 0이면(처음엔) mecab 적용함=> tokeniaer 후 인덱싱
        if count == 0:
            mecab_str = "(mecab 적용)"
            mecab_contexts=[]
            for context in tqdm(contexts):
                temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
                sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
                mecab_contexts.append(sentence)

            print(f'*contexts_len:{len(mecab_contexts)}')  
            print(f'{mecab_contexts[0:3]}')

            # tokeniaer 후 인덱싱
            tokenized_corpus = [BM25tokenizer(doc) for doc in mecab_contexts]
        else:
            # 2번째는 그냥 인덱싱
            tokenized_corpus = [BM25tokenizer(doc) for doc in contexts]
            
        bm25 = BM25Okapi(tokenized_corpus)

        #print(f'bm25.doc_len:{bm25.doc_len}')
        #print(f'type(bm25.doc_freqs):{type(bm25.doc_freqs)}')
        
        bm5_predictions_list = []
     
        # 쿼리 후 get_scores 를 이용하여, scores를 구함.
        for idx, query in enumerate(user_query):
            
            # 처음에는 mecab적용해서 query문 전처리 함.
            if count == 0:
                tempq = mecab.morphs(query) 
                query = " ".join(tempq)
                
            # 쿼리에 따른 스코어 구함    
            tokenized_query = BM25tokenizer(query)
            doc_scores = bm25.get_scores(tokenized_query)

            # 정렬후 최대 스코어 search_k 만큼만 출력함
            top_lists = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)[:search_k]
            bm5_predictions_list.append([index + contextids[0] for index, score in top_lists])

        # 정답, 여기서는 contextid를 리스트로 만듬.
        ground_truths_list = df_questions['contextid'].values.tolist()

        # MPR 계산
        predictions_list = bm5_predictions_list  # 예측 결과 리스트
        bm25_ranks, bm25_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

         # BM25-MRR 출력
        logger.info(f'--------------------------------------------------------------------------')
        logger.info('*BM25-MRR{}:{:.4f}'.format(mecab_str, bm25_score))
        logger.info(f'*Ranks({len(bm25_ranks)}):{bm25_ranks[0:20]}')
        # 검색 못한 계슈
        logger.info(f'---------------------------------------------------------------------------')
        zero_count = 0
        for item in bm25_ranks:
            if item == 0:
                zero_count += 1

        logger.info('*BM25{} 검색실패계수: {}/{}({:.2f}%)'.format(mecab_str, zero_count, len(bm25_ranks), (zero_count/len(bm25_ranks))*100))
        logger.info(f'---------------------------------------------------------------------------')

  0%|          | 0/964 [00:00<?, ?it/s]

*contexts_len:964
['1989 년 2 월 15 일 여의도 농민 폭력 시위 를 주도 한 혐의 ( 폭력 행위 등 처벌 에 관한 법률 위반 ) 으로 지명 수배 되 었 다 . 1989 년 3 월 12 일 서울 지방 검찰청 공 안 부 는 임종석 의 사전 구속 영장 을 발부 받 았 다 . 같 은 해 6 월 30 일 평양 축전 에 임수경 을 대표 로 파견 하 여 국가 보안법 위반 혐의 가 추가 되 었 다 . 경찰 은 12 월 18 일 ~ 20 일 사이 서울 경희대 학교 에서 임종석 이 성명 발표 를 추진 하 고 있 다는 첩보 를 입수 했 고 , 12 월 18 일 오전 7 시 40 분 경 가스총 과 전자 봉 으로 무장 한 특공 조 및 대공 과 직원 12 명 등 22 명 의 사복 경찰 을 승용차 8 대 에 나누 어 경희대 학교 에 투입 했 다 . 1989 년 12 월 18 일 오전 8 시 15 분 경 서울 청량리 경찰서 는 호위 학생 5 명 과 함께 경희대 학교 학생회관 건물 계단 을 내려오 는 임종석 을 발견 , 검거 해 구속 을 집행 했 다 . 임종석 은 청량리 경찰서 에서 약 1 시간 동안 조사 를 받 은 뒤 오전 9 시 50 분 경 서울 장안동 의 서울 지방 경찰청 공안 분실 로 인계 되 었 다 .', '" 내각 과 장관 들 이 소외 되 고 대통령 비서실 의 권한 이 너무 크 다 ", " 행보 가 비서 본연 의 역할 을 벗어난다 " 는 의견 이 제기 되 었 다 . 대표 적 인 예 가 10 차 개헌안 발표 이 다 . 원로 헌법 학자 인 허영 경희대 석좌 교수 는 정부 의 헌법 개정안 준비 과정 에 대해 " 청와대 비서실 이 아닌 국무회의 중심 으로 이뤄졌 어야 했 다 " 고 지적 했 다 . \' 국무 회의 의 심의 를 거쳐야 한다 \'( 제 89 조 ) 는 헌법 규정 에 충실하 지 않 았 다는 것 이 다 . 그러 면서 " 법무부 장관 을 제쳐 놓 고 민정 수석 이 개정안 을 설명 하 는 게 이해 가 안 된다 " 고 지적 했 다 . 민정 수석 은 국회의원 에

2023-02-05 10:10:54,289 - MRR-test - INFO - --------------------------------------------------------------------------
2023-02-05 10:10:54,291 - MRR-test - INFO - *BM25-MRR(mecab 적용):0.9295
2023-02-05 10:10:54,292 - MRR-test - INFO - *Ranks(500):[0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
2023-02-05 10:10:54,293 - MRR-test - INFO - ---------------------------------------------------------------------------
2023-02-05 10:10:54,294 - MRR-test - INFO - *BM25(mecab 적용) 검색실패계수: 5/500(1.00%)
2023-02-05 10:10:54,296 - MRR-test - INFO - ---------------------------------------------------------------------------
2023-02-05 10:10:55,357 - MRR-test - INFO - --------------------------------------------------------------------------
2023-02-05 10:10:55,358 - MRR-test - INFO - *BM25-MRR:0.8143
2023-02-05 10:10:55,359 - MRR-test - INFO - *Ranks(500):[0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.